In [45]:
import glob
from sam.stress_addition_model import (
    sam_prediction,
    Predicted_LCs,
    get_sam_lcs,
    SAM_Setting,
    OLD_STANDARD,
    Transforms
)
from sam.helpers import compute_lc, find_lc_99_max, compute_lc_from_curve
from sam.plotting import plot_sam_prediction
from sam.data_formats import ExperimentData, read_data, load_datapoints
import os
import matplotlib.pyplot as plt
import pandas as pd
import argparse
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [46]:
SETTINGS = OLD_STANDARD


rows = []

for path, data, name, val in tqdm(load_datapoints()):

    main_fit, stress_fit, sam_sur, sam_stress, additional_stress = sam_prediction(
        data.main_series,
        val,
        data.meta,
        settings=SETTINGS,
    )

    lcs = get_sam_lcs(stress_fit=stress_fit, sam_sur=sam_sur, meta=data.meta)

    row = {
        "path": path,
        "stressor": name,
        "stress_lc10": lcs.stress_lc10,
        "stress_lc50": lcs.stress_lc50,
        "sam_lc10": lcs.sam_lc10,
        "sam_lc50": lcs.sam_lc50,
        "survival_max": data.meta.max_survival,
        "lc1":main_fit.lc1,
        "lc99":main_fit.lc99,
    }

    rows.append(row)

df = pd.DataFrame(rows)


  0%|          | 0/47 [00:00<?, ?it/s]

100%|██████████| 47/47 [00:08<00:00,  5.63it/s]


In [47]:
df.lc1 = df.lc1.fillna(0)
normed = df.copy()
cols = ["stress_lc10", "stress_lc50", "sam_lc10", "sam_lc50"]
normed[cols] -= df.lc1.values[:,None]
normed[cols] = np.maximum(0,normed[cols])
normed[cols] /= (df.lc99.values[:,None] - df.lc1.values[:,None])
normed[cols].describe()

,stress_lc10,stress_lc50,sam_lc10,sam_lc50
count,23.000000,4.200000e+01,23.000000,4.200000e+01
mean,0.010363,4.551861e-02,0.004563,3.616506e-02
std,0.030241,7.129020e-02,0.013175,6.040465e-02
min,0.000000,9.465349e-08,0.000000,1.108104e-07
25%,0.000932,5.201214e-03,0.000000,3.627089e-03
50%,0.002438,1.947188e-02,0.000000,1.708471e-02
75%,0.007665,5.507705e-02,0.000486,4.547671e-02
max,0.146998,4.123040e-01,0.057608,3.452742e-01


In [48]:
fifty = normed.dropna(subset=["stress_lc50"])
mean_squared_error(fifty.stress_lc50, fifty.sam_lc50), mean_absolute_error(fifty.stress_lc50, fifty.sam_lc50)

(0.0006278829503431632, 0.014733547502801253)

In [49]:
ten = normed.dropna(subset=["stress_lc10"])
mean_squared_error(ten.stress_lc10, ten.sam_lc10), mean_absolute_error(ten.stress_lc10, ten.sam_lc10)

(0.0003714926038351805, 0.0070181567722982854)